In [ ]:
#global path
global_drive_path="/content/drive/MyDrive/ForeCache/interactions/"
          

In [ ]:
import os
import pandas as pd
import json
from pandas.io.json import json_normalize
from tqdm import tqdm


#for reward
import time
import datetime

pd.options.mode.chained_assignment = None  # default='warn'


In [ ]:
def clean_files(filePath,fileName):

  lastline = None
 # filePath="/content/drive/Shareddrives/Colearning-Exploration/Dataset/forecache_interactions_2016/forecache-interaction-analysis/forecache-interaction-analysis/interactions/taskname_ndsi-2d-subtask_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.txt"
  with open(filePath,"r") as f:
      lineList = f.readlines()
      lastline=lineList[-1]
  #with open(filePath,"r") as f, open(global_drive_path+"cleanInteractions/cleanInteractions-ndsi-2d-task/"+(str(fileName)+".json"),"w") as g:
  #with open(filePath,"r") as f, open(global_drive_path+"cleanInteractions/cleanInteractions-ndsi-3d-task/"+(str(fileName)+".json"),"w") as g:
  #os.makedirs(os.path.dirname(global_drive_path+"cleanInteractions/cleanInteractions-rgb-hurricane-3d-task/"+(str(fileName)+".json")))
  with open(filePath,"r") as f, open(global_drive_path+"cleanInteractions/cleanInteractions-rgb-hurricane-3d-task/"+(str(fileName)+".json"),"w+") as g:
   
      for i,line in enumerate(f,0):
          if i == 0:
              line = "["+str(line)+","
              g.write(line)
          elif line == lastline:            
              g.write(line)
              g.write("]")
          else:
              line = str(line)+","
              g.write(line)
          

In [ ]:
#directory =(str(global_drive_path) +"/ndsi-2d-task")
#directory =(str(global_drive_path) +"/ndsi-3d-task")
directory =(str(global_drive_path) +"/rgb-hurricane-3d-task")
failed_files=[]
for filename in os.listdir(directory):
  try:
    if filename.endswith(".txt"):
        
        filePath=(os.path.join(directory, filename))
        print(filename)
        name=os.path.splitext(filename)[0]
        clean_files(filePath,name)
    else:
        continue
  except OSError as err:
    print("OS error: {0}".format(err))
    failed_files.append(filename)
  except UnicodeDecodeError:
    print("Unicode Error")
    failed_files.append(filename)
  except BaseException as err:
    failed_files.append(filename)
    
    raise


taskname_rgb-hurricane-3d-task_userid_a6aab5f5-fdb6-41df-9fc6-221d70f8c6e8.txt
taskname_rgb-hurricane-3d-task_userid_bda49380-37ad-41c5-a109-7fa198a7691a.txt
taskname_rgb-hurricane-3d-task_userid_d6555293-35c7-4c27-b522-38d67d2d1b1a.txt
taskname_rgb-hurricane-3d-task_userid_cd3ef507-e3e9-4edb-9222-90a69a9bf272.txt
taskname_rgb-hurricane-3d-task_userid_954edb7c-4eae-47ab-9338-5c5c7eccac2d.txt
taskname_rgb-hurricane-3d-task_userid_ac5b65d7-f260-4534-9fae-d998b726ed32.txt
taskname_rgb-hurricane-3d-task_userid_da52acdd-3cea-483c-86e5-2c2740d371aa.txt
taskname_rgb-hurricane-3d-task_userid_bc9f9425-e867-4876-a5d9-791229916767.txt
taskname_rgb-hurricane-3d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.txt
taskname_rgb-hurricane-3d-task_userid_93be0a02-3f7a-46c9-b526-aa956ed00856.txt
taskname_rgb-hurricane-3d-task_userid_733a1ac5-0b01-485e-9b29-ac33932aa240.txt
taskname_rgb-hurricane-3d-task_userid_7d014c75-453b-4188-9e4e-fb086984896a.txt
taskname_rgb-hurricane-3d-task_userid_8bf5b71b-dfd4-

In [ ]:
#directory =(str(global_drive_path) +"/cleanInteractions//cleanInteractions-ndsi-3d-task")
#directory =(str(global_drive_path) +"/cleanInteractions//cleanInteractions-ndsi-2d-task")
directory =(str(global_drive_path) +"/cleanInteractions//cleanInteractions-rgb-hurricane-3d-task")
fileNames=[]
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        filePath=(os.path.join(directory, filename))
        fileNames.append(filename)
    else:
        continue

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def angle_between(p1, p2):
    ang1 = np.arctan2(*p1[::-1])
    ang2 = np.arctan2(*p2[::-1])
    return np.rad2deg((ang1 - ang2) % (2 * np.pi))

In [ ]:
def get_Angle(df):
  interactionData= pd.json_normalize(df.interactionData)
  x1=[]
  x2=[]
  y1=[]
  y2=[]
  angle=[]
  for i in range(len(interactionData['interactionType'])):
    #handles if the interactiontype doesn't have a coordinate : if interactionType is zoom
    if ((isinstance(interactionData['state.dragStart'][i], list) or isinstance(interactionData['state.dragEnd'][i], list) )==False):
      
      interactionData['state.dragStart'][i]=[0,0]
      interactionData['state.dragEnd'][i]=[0,0]
    #get between drag start and drag end
    angle.append(angle_between(interactionData['state.dragStart'][i],interactionData['state.dragEnd'][i]))
    #print(angle)

    #this are to plot later on
    x1.append(interactionData['state.dragStart'][i][0])
    y1.append(interactionData['state.dragStart'][i][1])

    x2.append(interactionData['state.dragEnd'][i][0])
    y2.append(interactionData['state.dragEnd'][i][1])
  return angle



In [ ]:
#to generate reward based on radiclick and snapshot data 
def get_Reward(dataframe,filenamed):
  #global path
  reward=[]
  subtask=[]
  raw_timestamp= dataframe.timestampMillis
  snapshot_global_drive_path="/content/drive/MyDrive/ForeCache/forecache_user_study_data_dump/forecache_user_study_data_dump"
  #get a pandas dataframe object from json file
  path= snapshot_global_drive_path + "/snapshots.json"
  with open(path) as project_file:    
      data = json.load(project_file) 
  df_snapshots = pd.json_normalize(data)

  #get userid from fileName
  #filenamed[filenamed.find("userid_")+7: filenamed.find(".csv")] need to remove  hard code later
  dataframe_dimension=df_snapshots.loc[(df_snapshots['taskname']=="rgb-hurricane-3d-task") & (df_snapshots['userid']==filenamed[filenamed.find("userid_")+7: filenamed.find(".json")])] #change here for different Task name 2d, 3d
  print("There are ", len(dataframe_dimension) , "Snapshot times")
  for i in tqdm(range(len(raw_timestamp))):
      stringmyDate=str(raw_timestamp[i])
      myDate=stringmyDate[: stringmyDate.find(".")]
      dt = datetime.datetime.strptime(myDate, "%Y-%m-%d %H:%M:%S")
      a=time.mktime(dt.timetuple()) + (dt.microsecond / 1000000.0)
      temp=0
      task=1
      for j in range(len(dataframe_dimension["timestampMillis"])):


        if abs(a - (dataframe_dimension.timestampMillis.iloc[j]/1000)) < 10:
          temp=temp+10
        else:
         temp=temp+0
        if (a - (dataframe_dimension.timestampMillis.iloc[j]/1000)) < 0:
          task=task
        else:
          task=task+1
      subtask.append(task)
      reward.append(temp)
  return reward, subtask
    

In [ ]:
def get_Zoom(df_zoom):
  df_zoom['state.newZoom'] = df_zoom['state.newZoom'].replace(np.nan, 0)
  lastZoom=[0,0]
  zoom_level=[]
  for i in tqdm(range(len(df_zoom))):
    if df_zoom['state.newZoom'][i]==0:
      if (lastZoom!=[0,0]):
          df_zoom['state.newZoom'][i]=lastZoom
      else:
        df_zoom['state.newZoom'][i]= [0,0]
        lastZoom= [0,0]
    else:
      lastZoom = df_zoom['state.newZoom'][i]
    zoom_level.append(df_zoom['state.newZoom'][i][0])
  return zoom_level

In [ ]:
def get_IdleTime(df):
  idle_time=[]
  previous_time=df['timestampMillis'][0]
  for i in tqdm(range(len(df))):
      wait_time=df['timestampMillis'][i]-previous_time
      previous_time=df['timestampMillis'][i]


      #fixing format
      idle_time.append(wait_time.total_seconds())
  return idle_time

In [ ]:
import heapq
def get_Subset_Idletime(idle_time):
  subset_idle_time=[]
  largest_indexes=heapq.nlargest(2, range(len(idle_time)), key=idle_time.__getitem__)
  second_max_index=largest_indexes[1]
  for j in tqdm(range(len(idle_time))):
    if (j<= second_max_index):
      subset_idle_time.append(1)
    else:
      subset_idle_time.append(2)
  return subset_idle_time

In [ ]:
def get_State(df_state):
  states_array=[]
  for i in range(len(df_state)):
    if pd.isna(df_state.phase[i]):
      states_array.append('Answering')
    else:
      states_array.append(df_state.phase[i])
  return states_array

In [ ]:
# from collections import defaultdict
# probabilities=[]
# states_actions=[]
# _map = defaultdict(int)
# states=['Answering','SenseMaking','Foraging','Navigation']
# actions=['start','pan','zoom','radioClick']
# for state in states:
#   for action in actions:
#     states_actions.append(state+action)
#     _map[state+action]=1


In [ ]:
from collections import defaultdict
def get_probabilities(df):
    probabilities=[]
    map=dict()

    states=['Answering','Sensemaking','Foraging','Navigation']
    actions=['start','pan','zoom','radioClick']
    for state in states:
      for action in actions:
        map[state+action]=1


    for i in range(len(df)):
      #print(str(df['State'][i]+df['Action']))
      if df['State'][i]=="Navigation":
        map[str(df['State'][i]+df['Action'][i])] += 1
        probs = (map[str(df['State'][i]+df['Action'][i])])/(map['Navigationpan'] + map['NavigationradioClick']+ map['Navigationstart'] + map['Navigationzoom'])

      elif df['State'][i]=="Sensemaking":
        map[str(df['State'][i]+df['Action'][i])] += 1
        probs = (map[str(df['State'][i]+df['Action'][i])])/(map['Sensemakingpan'] + map['SensemakingradioClick']+ map['Sensemakingstart'] + map['Sensemakingzoom'])
      elif df['State'][i]=="Foraging":
        map[str(df['State'][i]+df['Action'][i])] += 1
        probs = (map[str(df['State'][i]+df['Action'][i])])/(map['Foragingpan'] + map['ForagingradioClick']+ map['Foragingstart'] + map['Foragingzoom'])
      else:
        map[str(df['State'][i]+df['Action'][i])] += 1
        probs = (map[str(df['State'][i]+df['Action'][i])])/(map['Answeringpan'] + map['AnsweringradioClick']+ map['Answeringstart'] + map['Answeringzoom'])
      probabilities.append(probs)
    return probabilities

MAIN FUNCTION THAT RETURNS THE COMPLETED FILE

In [ ]:
error_files=[]

for filename in tqdm(fileNames):
  try:
    #saveDir=(str(global_drive_path) +"/stateActionModel/ndsi-2d")
    #saveDir=(str(global_drive_path) +"/stateActionModel/ndsi-3d")
    saveDir=(str(global_drive_path) +"/stateActionModel/rgb-hurricane-3d")


    #get a pandas dataframe object from json file
    df = pd.read_json(os.path.join(directory,filename))
    print("Read file:",filename)
    interactionData=pd.json_normalize(df.interactionData)
    final_dataframe = pd.DataFrame(columns=['Action','State','TimeStamp','StateActionProbab','IdleTime','Reward','Subtask_Screenshot_Time','Subtask_Idle_Time','ZoomLevel'])
    final_dataframe['Action']=interactionData.interactionType
    final_dataframe['State']=get_State(df)
    final_dataframe['TimeStamp']=df.timestampMillis
    final_dataframe['IdleTime']=get_IdleTime(df)
    final_dataframe['Subtask_Idle_Time']= get_Subset_Idletime(final_dataframe['IdleTime'])
    final_dataframe['Reward'],final_dataframe['Subtask_Screenshot_Time']= get_Reward(df,filename)
    final_dataframe['Angle']= get_Angle(df)
    final_dataframe['ZoomLevel']= get_Zoom(interactionData)	
    #get probabilities
    final_dataframe['StateActionProbab']=get_probabilities(final_dataframe)
    txt_version= filename.replace("json","csv")
    final_dataframe.to_csv(os.path.join(saveDir,txt_version), index=False)
  except AttributeError:
    print("AttributeError")
    error_files.append(filename)


  0%|          | 0/20 [00:00<?, ?it/s]

Read file: taskname_rgb-hurricane-3d-task_userid_a6aab5f5-fdb6-41df-9fc6-221d70f8c6e8.json



100%|██████████| 68/68 [00:00<00:00, 11169.92it/s]

100%|██████████| 68/68 [00:00<00:00, 384383.65it/s]


There are  2 Snapshot time



100%|██████████| 68/68 [00:00<00:00, 4036.92it/s]

  5%|▌         | 1/20 [00:01<00:27,  1.46s/it]

...


100%|██████████| 45/45 [00:00<00:00, 2669.45it/s]

 95%|█████████▌| 19/20 [00:19<00:01,  1.01s/it]

Read file: taskname_rgb-hurricane-3d-task_userid_e4221e48-0930-40db-af3c-f4de1ed87582.json



100%|██████████| 74/74 [00:00<00:00, 26170.19it/s]

100%|██████████| 74/74 [00:00<00:00, 388458.69it/s]


There are  2 Snapshot time



100%|██████████| 74/74 [00:00<00:00, 3857.74it/s]

100%|██████████| 20/20 [00:20<00:00,  1.03s/it]


RANDOM TEST SNIPPETS AFTER THIS POINT

In [ ]:
len(matchingName) # so 19 users are the same 

NameError: ignored

In [ ]:
filenamed="taskname_ndsi-3d-task_userid_8b544d24-3274-4bb0-9719-fd2bccc87b02.csv"

In [ ]:
filenamed[filenamed.find("userid_")+7: filenamed.find(".csv")]

In [ ]:
df.head(5)

In [ ]:
df.phase

In [ ]:
interactionData= pd.io.json.json_normalize(df.interactionData)

In [ ]:
interactionData.interactionType

In [ ]:
final_dataframe = pd.DataFrame(columns=['Action','State','TimeStamp'])
final_dataframe['Action']=interactionData.interactionType
final_dataframe['State']=df.phase
final_dataframe['TimeStamp']=df.timestampMillis

In [ ]:
final_dataframe.head

In [ ]:
final_dataframe['Action']=interactionData.interactionType
final_dataframe['State']=df.phase
final_dataframe['TimeStamp']=df.timestampMillis

In [ ]:
final_dataframe

In [ ]:
for item in df.toFetch:
  print (item)

In [ ]:
toFetch

In [ ]:
import json
from pandas.io.json import json_normalize
normalized_json = json_normalize(df['taskname_ndsi-2d-subtask_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45'])
normalized_json.head(3 )


In [ ]:
normalized_json['toFetch']

In [ ]:
#get a pandas dataframe object from json file
#df = pd.read_json("/content/interactions_data/zoom_0_pos_0_0.json")
import json
import pandas as pd
from pandas.io.json import json_normalize
path= '/content/drive/Shareddrives/Colearning-Exploration/Dataset/forecache_interactions_2016/ndsi_2d_jsons/zoom_0_pos_0_0.json'
with open(path) as project_file:    
    data = json.load(project_file)  

df_zoom = pd.json_normalize(data)
df_zoom.head(5)

In [ ]:
df_zoom.domains